In [ ]:
!pip install pandas numpy requests scikit-learn imbalanced-learn xgboost tensorflow shap lime matplotlib imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 525.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 72.1 MB/s eta 0:00:0

# Step 1: Data Acquisition

In [ ]:
import requests
import pandas as pd
import numpy as np

ETHERSCAN_API_KEY = "V8RHS7P2YNSAHUY92CXVANVQK8MIYK95UQ"
BASE_URL = "https://api.etherscan.io/api"

def get_transactions(address, start_block=0, end_block=99999999):
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': address,
        'startblock': start_block,
        'endblock': end_block,
        'sort': 'asc',
        'apikey': ETHERSCAN_API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    return response.json()['result']

normal_addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "0xDC76CD25977E0a5Ae17155770273aD58648900D3",
    "0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0",
]

fraud_addresses = [
    "0x283aa3c6e0cf2c2d8f2c1c3b7603e7b4c8a9f2a6",
    "0x6f46cf5569aefa1acc1009290c8e043747172d89",
]

normal_txns = [get_transactions(addr) for addr in normal_addresses]
fraud_txns = [get_transactions(addr) for addr in fraud_addresses]

normal_df = pd.DataFrame([tx for sublist in normal_txns for tx in sublist])
normal_df['is_fraud'] = 0
fraud_df = pd.DataFrame([tx for sublist in fraud_txns for tx in sublist])
fraud_df['is_fraud'] = 1
df = pd.concat([normal_df, fraud_df], axis=0)


# Step 2: Data Preprocessing & Feature Engineering

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime

def preprocess_data(df):
    # Remove non-numeric columns for ML
    non_numeric = ['hash', 'nonce', 'blockHash', 'from', 'to', 'input', 'contractAddress', 'cumulativeGasUsed', 'blockNumber', 'timeStamp', 'transactionIndex']
    for col in non_numeric:
        if col in df.columns:
            df = df.drop(columns=[col])
    # Convert timestamp first for features
    if 'timeStamp' in df.columns:
        df['timestamp'] = df['timeStamp'].apply(lambda x: datetime.fromtimestamp(int(x)))
    else:
        df['timestamp'] = pd.to_datetime('now')
    # Feature engineering
    df['value_eth'] = df['value'].astype(float) / 1e18
    df['gas_price_gwei'] = df['gasPrice'].astype(float) / 1e9
    df['gas_used'] = df['gasUsed'].astype(float)
    df['gas_cost'] = df['gas_price_gwei'] * df['gas_used']
    df['hour_of_day'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['value_gas_ratio'] = df['value_eth'] / (df['gas_cost'] + 1e-9)
    # Add sender/receiver txn count (optional, but requires original from/to columns)
    df['sender_txn_count'] = 1  # Dummy if dropped
    df['receiver_txn_count'] = 1
    # Select features
    features = [
        'value_eth', 'gas_price_gwei', 'gas_used', 'gas_cost',
        'hour_of_day', 'day_of_week', 'is_weekend', 'value_gas_ratio',
        'sender_txn_count', 'receiver_txn_count'
    ]
    X = df[features]
    y = df['is_fraud']
    return X, y

X, y = preprocess_data(df)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# Step 3: Class Balancing

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.1, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
resample_pipeline = Pipeline([
    ('o', over),
    ('u', under)
])
X_train_res, y_train_res = resample_pipeline.fit_resample(X_train, y_train)


# Step 4: Model Building

## XGBoost Model


In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    scale_pos_weight=(len(y_train_res) - sum(y_train_res)) / sum(y_train_res)
)
xgb_model.fit(X_train_scaled, y_train_res)
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]
print("XGBoost Performance:")
print(classification_report(y_test, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_xgb))


XGBoost Performance:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      4159
           1       0.36      0.96      0.52       100

    accuracy                           0.96      4259
   macro avg       0.68      0.96      0.75      4259
weighted avg       0.98      0.96      0.97      4259

ROC-AUC: 0.9846585717720606


## LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam

# We create sequences using only numeric features
def create_sequences(X, y, sequence_length=10):
    sequences, labels = [], []
    for i in range(len(X) - sequence_length):
        seq = X.iloc[i:i+sequence_length].values
        label = y.iloc[i+sequence_length-1]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

X_sequences, y_sequences = create_sequences(X, y)
X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42, stratify=y_sequences
)
# Resample: flatten to 2D, resample, then reshape
X_seq_train_2d = X_seq_train.reshape(X_seq_train.shape[0], -1)
X_seq_train_res, y_seq_train_res = resample_pipeline.fit_resample(X_seq_train_2d, y_seq_train)
X_seq_train_res = X_seq_train_res.reshape(-1, X_seq_train.shape[1], X_seq_train.shape[2])

# Scale
seq_scaler = StandardScaler()
X_seq_train_res_flat = X_seq_train_res.reshape(-1, X_seq_train_res.shape[2])
X_seq_train_scaled = seq_scaler.fit_transform(X_seq_train_res_flat).reshape(X_seq_train_res.shape)
X_seq_test_flat = X_seq_test.reshape(-1, X_seq_test.shape[2])
X_seq_test_scaled = seq_scaler.transform(X_seq_test_flat).reshape(X_seq_test.shape)

# LSTM Model
lstm_model = Sequential([
    Masking(mask_value=0., input_shape=(X_seq_train_scaled.shape[1], X_seq_train_scaled.shape[2])),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
lstm_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_lstm = lstm_model.fit(
    X_seq_train_scaled, y_seq_train_res,
    validation_data=(X_seq_test_scaled, y_seq_test),
    epochs=10,
    batch_size=64,
    class_weight={0: 1., 1: 5.}
)
y_pred_lstm = (lstm_model.predict(X_seq_test_scaled) > 0.5).astype(int)
y_proba_lstm = lstm_model.predict(X_seq_test_scaled)
print("LSTM Performance:")
print(classification_report(y_seq_test, y_pred_lstm))
print("ROC-AUC:", roc_auc_score(y_seq_test, y_proba_lstm))



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.5162 - loss: 1.2226 - val_accuracy: 0.8553 - val_loss: 0.4440
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9002 - loss: 0.5433 - val_accuracy: 0.8818 - val_loss: 0.4117
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8984 - loss: 0.5194 - val_accuracy: 0.9079 - val_loss: 0.3131
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9155 - loss: 0.4478 - val_accuracy: 0.8919 - val_loss: 0.3314
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9127 - loss: 0.3922 - val_accuracy: 0.9288 - val_loss: 0.2462
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9342 - loss: 0.3407 - val_accuracy: 0.9300 - val_loss: 0.1906
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9395 - loss: 0.3045 - val_accuracy: 0.9354 - val_loss: 0.2123
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9464 - loss: 0.2637 - val_accuracy: 0.9509 - val_loss: 0.

## CNN Model

In [ ]:
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_seq_train_scaled.shape[1], X_seq_train_scaled.shape[2])),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_cnn = cnn_model.fit(
    X_seq_train_scaled, y_seq_train_res,
    validation_data=(X_seq_test_scaled, y_seq_test),
    epochs=10,
    batch_size=64,
    class_weight={0: 1., 1: 5.}
)
y_pred_cnn = (cnn_model.predict(X_seq_test_scaled) > 0.5).astype(int)
y_proba_cnn = cnn_model.predict(X_seq_test_scaled)
print("CNN Performance:")
print(classification_report(y_seq_test, y_pred_cnn))
print("ROC-AUC:", roc_auc_score(y_seq_test, y_proba_cnn))



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5540 - loss: 1.3734 - val_accuracy: 0.2206 - val_loss: 0.8929
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5784 - loss: 0.9334 - val_accuracy: 0.8325 - val_loss: 0.4880
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8959 - loss: 0.5475 - val_accuracy: 0.9274 - val_loss: 0.2706
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9317 - loss: 0.3799 - val_accuracy: 0.9535 - val_loss: 0.1737
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9490 - loss: 0.2972 - val_accuracy: 0.9568 - val_loss: 0.1508
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9589 - loss: 0.2438 - val_accuracy: 0.9307 - val_loss: 0.2239
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9545 - loss: 0.2273 - val_accuracy: 0.9652 - val_loss: 0.1058
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9658 - loss: 0.2080 - val_accuracy: 0.9641 - val_loss: 0.1073
Epo

## GCN Model

In [ ]:
!pip install torch_geometric

  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import LabelEncoder

if 'value_eth' not in df.columns:
    df['value_eth'] = df['value'].astype(float) / 1e18

# Build mapping for addresses
addresses = pd.concat([df['from'], df['to']]).unique()
addr2idx = {addr: idx for idx, addr in enumerate(addresses)}

edges = torch.tensor([
    [addr2idx[f], addr2idx[t]]
    for f, t in zip(df['from'], df['to'])
    if f in addr2idx and t in addr2idx
], dtype=torch.long).t().contiguous()

feat_df = pd.DataFrame({'address': addresses})
feat_df['sent_count'] = feat_df['address'].map(df['from'].value_counts()).fillna(0)
feat_df['recv_count'] = feat_df['address'].map(df['to'].value_counts()).fillna(0)
feat_df['sent_value'] = feat_df['address'].map(df.groupby('from')['value_eth'].sum()).fillna(0)
feat_df['recv_value'] = feat_df['address'].map(df.groupby('to')['value_eth'].sum()).fillna(0)
x = torch.tensor(feat_df[['sent_count','recv_count','sent_value','recv_value']].values, dtype=torch.float)

# -- Assign node labels, ensuring more than one fraud label if possible --
feat_df['label'] = feat_df['address'].apply(lambda x: 1 if x in fraud_addresses else 0)
y = torch.tensor(feat_df['label'].values, dtype=torch.long)

# -- Make sure you have enough fraud nodes for training --
print('Fraud nodes in graph:', y.sum().item())
if y.sum().item() < 2:
    print('Not enough fraud samples for GCN to learn! Add more fraud addresses if possible.')

# -- Train/test mask: stratified split for node classification --
from sklearn.model_selection import train_test_split
idx = np.arange(len(addresses))
train_idx, test_idx = train_test_split(
    idx, test_size=0.2, stratify=y, random_state=42
)
train_mask = torch.zeros(len(addresses), dtype=torch.bool)
train_mask[train_idx] = True
test_mask = torch.zeros(len(addresses), dtype=torch.bool)
test_mask[test_idx] = True

# -- Define a weighted loss for class imbalance --
weights = torch.tensor([1.0, float((y == 0).sum()) / float((y == 1).sum() + 1e-8)])
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

# -- GCN Definition --
class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleGCN(x.shape[1], 16, 2).to(device)
x, edges, y = x.to(device), edges.to(device), y.to(device)
train_mask, test_mask = train_mask.to(device), test_mask.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(1, 51):
    model.train()
    optimizer.zero_grad()
    out = model(x, edges)
    loss = loss_fn(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# -- Evaluate --
model.eval()
with torch.no_grad():
    logits = model(x, edges)
    pred = logits.argmax(dim=1)
    probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
    y_true = y[test_mask].cpu().numpy()
    y_pred = pred[test_mask].cpu().numpy()
    try:
        auc = roc_auc_score(y_true, probs[test_mask.cpu().numpy()])
    except:
        auc = None
    print('GCN Classification Report:\n', classification_report(y_true, y_pred, digits=4))
    if auc is not None:
        print('GCN ROC-AUC: %.6f' % auc)
    else:
        print('GCN ROC-AUC: undefined (only one class present)')

# -- Explainability with GNNExplainer (local explanation for one node) --
from torch_geometric.nn import GNNExplainer
explainer = GNNExplainer(model, epochs=100)
# Pick a fraud node for explanation
fraud_nodes = np.where(y.cpu().numpy() == 1)[0]
if len(fraud_nodes) > 0:
    node_idx = int(fraud_nodes[0])
    node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edges)
    print("Top node features importance (fraud node):")
    print(node_feat_mask.cpu().detach().numpy())
    explainer.visualize_subgraph(node_idx, edges.cpu(), edge_mask.cpu(), y.cpu())
else:
    print("No fraud nodes found for GNN explanation.")

Fraud nodes in graph: 1
Not enough fraud samples for GCN to learn! Add more fraud addresses if possible.


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

# Step 5: Model Evaluation

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

def plot_roc_curve(y_true, y_proba, model_name, color=None):
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})', color=color)
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')

def plot_pr_curve(y_true, y_proba, model_name, color=None):
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    ap = average_precision_score(y_true, y_proba)
    plt.plot(recall, precision, label=f'{model_name} (AP = {ap:.2f})', color=color)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='upper right')

# === Main plotting section ===

plt.figure(figsize=(10, 8))
plot_roc_curve(y_test, y_proba_xgb, 'XGBoost', color='C0')
plot_roc_curve(y_seq_test, y_proba_lstm, 'LSTM', color='C1')
plot_roc_curve(y_seq_test, y_proba_cnn, 'CNN', color='C2')
# Add GCN if available
if 'gcn_probs' in globals() and 'test_mask' in globals():
    # Use only test_mask indices for evaluation
    gcn_y_true = y[test_mask].cpu().numpy() if hasattr(y[test_mask], 'cpu') else y[test_mask].values
    gcn_y_proba = gcn_probs[test_mask] if hasattr(gcn_probs, '__getitem__') else gcn_probs
    plot_roc_curve(gcn_y_true, gcn_y_proba, 'GCN (GNN)', color='C3')
plt.show()

plt.figure(figsize=(10, 8))
plot_pr_curve(y_test, y_proba_xgb, 'XGBoost', color='C0')
plot_pr_curve(y_seq_test, y_proba_lstm, 'LSTM', color='C1')
plot_pr_curve(y_seq_test, y_proba_cnn, 'CNN', color='C2')
if 'gcn_probs' in globals() and 'test_mask' in globals():
    plot_pr_curve(gcn_y_true, gcn_y_proba, 'GCN (GNN)', color='C3')
plt.show()

# Step 6: Explainability with SHAP and LIME

# SHAP Analysis

In [ ]:
import shap
explainer_xgb = shap.TreeExplainer(xgb_model)
shap_values_xgb = explainer_xgb.shap_values(X_test_scaled)
shap.summary_plot(shap_values_xgb, X_test_scaled, feature_names=X.columns)
shap.force_plot(explainer_xgb.expected_value, shap_values_xgb[0, :], X_test_scaled[0, :], feature_names=X.columns)


## LIME Analysis



In [ ]:
import lime
import lime.lime_tabular
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    X_train_scaled,
    feature_names=X.columns,
    class_names=['Normal', 'Fraud'],
    mode='classification'
)
exp = explainer_lime.explain_instance(
    X_test_scaled[0],
    xgb_model.predict_proba,
    num_features=10
)
exp.show_in_notebook()

# Step 7: Comparison & Reporting

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

def evaluate_model(y_true, y_pred, y_proba, model_name):
    return {
        'Model': model_name,
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, zero_division=0),
        'ROC-AUC': roc_auc_score(y_true, y_proba)
    }

# Gather results
results = [
    evaluate_model(y_test, y_pred_xgb, y_proba_xgb, 'XGBoost'),
    evaluate_model(y_seq_test, y_pred_lstm, y_proba_lstm, 'LSTM'),
    evaluate_model(y_seq_test, y_pred_cnn, y_proba_cnn, 'CNN')
]

# Add GCN/GNN if available
if 'gcn_probs' in globals() and 'test_mask' in globals():
    gcn_y_true = y[test_mask].cpu().numpy() if hasattr(y[test_mask], 'cpu') else y[test_mask].values
    gcn_y_pred = (gcn_probs[test_mask] > 0.5).astype(int) if hasattr(gcn_probs, '__getitem__') else (gcn_probs > 0.5).astype(int)
    gcn_y_proba = gcn_probs[test_mask] if hasattr(gcn_probs, '__getitem__') else gcn_probs
    results.append(evaluate_model(gcn_y_true, gcn_y_pred, gcn_y_proba, 'GCN (GNN)'))

results_df = pd.DataFrame(results)

print("Model Performance Comparison:")
print(results_df)

# Visual comparison
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
for i, metric in enumerate(metrics):
    plt.subplot(1, 5, i+1)
    plt.bar(results_df['Model'], results_df[metric])
    plt.title(metric)
    plt.ylim(0, 1)
    plt.xticks(rotation=20)
plt.tight_layout()
plt.show()